In [ ]:
!pip install --upgrade gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

wb = gc.open_by_key('1Is-380wsfiKOzSR7p-TWEQkPFeUlXViq0xc1rFMH_GU')
ws = wb.worksheet('datos')

# get_all_values gives a list of rows.
rows = ws.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)
print(df)


               0        1        2 3                        4
0    Temperatura  presion  Altitud     ID de la celda proxima
1           28.3   852.04  1437.61                        140
2          28.37    852.1  1437.91                           
3          28.43   852.03  1438.29                           
4          31.62   852.45  1433.12                           
..           ...      ...      ... ..                     ...
136        29.12   862.77   1327.1                           
137        28.85   857.22   1388.4                           
138        22.17   854.87  1410.73                           
139        22.17   854.82   1411.0                           
140        26.14   854.83  1409.95                           

[141 rows x 5 columns]


In [12]:
from google.colab import auth
auth.authenticate_user()

import numpy as np
from numba import cuda
import gspread
import math
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

wb = gc.open_by_key('1Is-380wsfiKOzSR7p-TWEQkPFeUlXViq0xc1rFMH_GU')
ws = wb.worksheet('DatosTransformar')

# get_all_values gives a list of rows.
rows = ws.get_all_values()

@cuda.jit
def celcius_kelvin(y,outK,outF ):
  indx = cuda.grid(1)
  outK[indx] = y[indx] + 273.15
  outF[indx] = (1.8 * y[indx]) + 32

#main
array = np.array(ws.get_all_values())
arrayFloat = []
for i in range(array.size):
  arrayFloat.append(float(array[i][0]))

# Traslado de memoria de host a device
d_a = cuda.to_device(arrayFloat)
# creamos la salida en el device
d_out_K = cuda.device_array_like(d_a)
d_out_F = cuda.device_array_like(d_a)

# usamos formula para sacar los bloques, cada uno con 256 threads
N = 10000
threads_per_block = 256
blocks_per_grid = (N // threads_per_block) + 1
celcius_kelvin[blocks_per_grid, threads_per_block](d_a, d_out_K, d_out_F)
# wait for all threads to complete
cuda.synchronize()
# copia de informacion de device a host
# e impresion de ello
impresionC = d_a.copy_to_host()
h_out_K = d_out_K.copy_to_host()
h_out_F = d_out_F.copy_to_host()
for i in range(h_out_K.size):
  print("Conversion: " + str(impresionC[i])+"C"+" a " + str(round(h_out_K[i],2)) + "K" + " - " + str(round(h_out_F[i],2)) + "F" )


Conversion: 28.3C a 301.45K - 82.94F
Conversion: 28.37C a 301.52K - 83.07F
Conversion: 28.43C a 301.58K - 83.17F
Conversion: 31.62C a 304.77K - 88.92F
Conversion: 33.09C a 306.24K - 91.56F
Conversion: 33.63C a 306.78K - 92.53F
Conversion: 33.97C a 307.12K - 93.15F
Conversion: 34.24C a 307.39K - 93.63F
Conversion: 34.65C a 307.8K - 94.37F
Conversion: 34.77C a 307.92K - 94.59F
Conversion: 34.86C a 308.01K - 94.75F
Conversion: 34.93C a 308.08K - 94.87F
Conversion: 35.0C a 308.15K - 95.0F
Conversion: 35.04C a 308.19K - 95.07F
Conversion: 35.07C a 308.22K - 95.13F
Conversion: 35.17C a 308.32K - 95.31F
Conversion: 28.88C a 302.03K - 83.98F
Conversion: 28.97C a 302.12K - 84.15F
Conversion: 26.91C a 300.06K - 80.44F
Conversion: 26.91C a 300.06K - 80.44F
Conversion: 26.91C a 300.06K - 80.44F
Conversion: 26.91C a 300.06K - 80.44F
Conversion: 27.06C a 300.21K - 80.71F
Conversion: 29.23C a 302.38K - 84.61F
Conversion: 30.07C a 303.22K - 86.13F
Conversion: 30.69C a 303.84K - 87.24F
Conversion: 31.0

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 40 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
